In [189]:
#Current imports
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler

##Tensorflow
import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Input, Flatten, Dropout



In [190]:
##Read in CSV
df = pd.read_csv('IMDB_Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [191]:
y=df['sentiment'].map({'positive': 1, 'negative': 0})
##turn string negative to positive into binary classification
print(y.head())

reviews=df['review']
print(reviews.head())




0    1
1    1
2    1
3    0
4    1
Name: sentiment, dtype: int64
0    One of the other reviewers has mentioned that ...
1    A wonderful little production. <br /><br />The...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
Name: review, dtype: object


In [192]:
##Preprocessing libraries
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [193]:
##Preprocessing code
##remove stop words? ##negligible
##lowercase or standardize the format or each word? 
## case sensitivity might matter in sentiment analysis


tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
word_index = tokenizer.word_index
padded_sequences = pad_sequences(sequences)


In [194]:
print(padded_sequences)
print(padded_sequences.shape)
##50,000 sequences padded to a length of 2493

[[    0     0     0 ...   125  4103   486]
 [    0     0     0 ...  1977    69   221]
 [    0     0     0 ...    63    16   350]
 ...
 [    0     0     0 ... 22840     2  6050]
 [    0     0     0 ...    67   739    42]
 [    0     0     0 ...   794    11    17]]
(50000, 2493)


In [195]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.4, random_state=22)

In [202]:
##Experiment with this
EMBEDDING_DIM = 50

In [197]:
##Testing environment
print(Input(shape=(padded_sequences.shape[1],)))
print(Input(shape=(X_train.shape[1],)))


KerasTensor(type_spec=TensorSpec(shape=(None, 2493), dtype=tf.float32, name='input_29'), name='input_29', description="created by layer 'input_29'")
KerasTensor(type_spec=TensorSpec(shape=(None, 2493), dtype=tf.float32, name='input_30'), name='input_30', description="created by layer 'input_30'")


In [204]:
def doc2vec_model():
    
    ##We have a 2D representation for the tensor, reduce to 1D which is what we need
    ##first dimension will have variable size and second dimension is 2493

    input_layer = Input(shape=(X_train.shape[1],))


    #Embedding layer
    ##word_index is a dictionary, add +1 for the padding token
    embedding_layer = Embedding(input_dim=len(word_index) + 1, output_dim=EMBEDDING_DIM)(input_layer)

    #GlobalAveragePooling1D layer
    ##average_pooling = GlobalAveragePooling1D()(embedding_layer)

    #Flattening
    flatten = Flatten()(embedding_layer)

    # Additional Dense layers with dropout for regularization
    dense_1 = Dense(64, activation='relu')(flatten)
    dropout_1 = Dropout(0.2)(dense_1)

    dense_2 = Dense(32, activation='relu')(dropout_1)
    dropout_2 = Dropout(0.1)(dense_2)


    #Dense layers
    #Optimization such as dropout??

    output_layer = Dense(1, activation='sigmoid')(dropout_2)


    model = Model(inputs=input_layer, outputs=output_layer)

    #binary or multi-class compilation
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [207]:
##Train actual model
model = doc2vec_model()
model.fit(X_train, y_train, batch_size=10, epochs=2, verbose=1)


Epoch 1/2
3000/3000 [==============================] - 310s 103ms/step - loss: 0.4701 - accuracy: 0.7380
Epoch 2/2
3000/3000 [==============================] - 320s 107ms/step - loss: 0.1618 - accuracy: 0.9425


In [208]:
# Evaluate the model on the test set
results = model.evaluate(X_test, y_test, batch_size=10)

# Print the evaluation results
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])

2000/2000 [==============================] - 12s 6ms/step - loss: 0.2780 - accuracy: 0.8910
Test Loss: 0.2779521644115448
Test Accuracy: 0.8910499811172485


In [179]:
import os

def read_txt_files(directory_path):
    file_contents = []

    # Iterate over each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            
            # Read the content of the file and append it to the list
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                file_contents.append(content)

    return file_contents

pos_list = read_txt_files('pos')
neg_list = read_txt_files('neg')

# Replace 'your_directory_path' with the path to your directory containing .txt files
list_of_strings = pos_list + neg_list




In [180]:
pos_list_bin = [1 for i in list(range(12500))]
neg_list_bin = [0 for i in list(range(12500))]


# Replace 'your_directory_path' with the path to your directory containing .txt files
list_of_strings = pos_list + neg_list
bin_list = pos_list_bin + neg_list_bin


In [236]:
tokenizer_test = Tokenizer()
tokenizer_test.fit_on_texts(list_of_strings)
sequences_test = tokenizer_test.texts_to_sequences(list_of_strings)
word_index_test = tokenizer_test.word_index
padded_test = pad_sequences(sequences_test, maxlen=2493)


In [182]:
bin_list = pd.Series(bin_list)

In [183]:
print(type(padded_sequences))
print(type(y))

print(type(padded_test))
print(type(bin_list))

<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


In [209]:
# Evaluate the model on the test set
# Assuming X_test is the input data
results = model.evaluate(padded_test, bin_list, batch_size=10)

# Print the evaluation results
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])

2500/2500 [==============================] - 14s 5ms/step - loss: 1.0749 - accuracy: 0.5534
Test Loss: 1.0748658180236816
Test Accuracy: 0.5534399747848511


In [252]:
##Preprocessing for new model
##we concatenate our already padded sequences of the same length along rows
combined_sequences = np.concatenate((padded_sequences, padded_test), axis=0)
print(combined_sequences)

[   0    0    0 ...  125 4103  486]
[[    0     0     0 ...   125  4103   486]
 [    0     0     0 ...  1977    69   221]
 [    0     0     0 ...    63    16   350]
 ...
 [    0     0     0 ...   125   332   154]
 [    0     0     0 ...    62   177     5]
 [    0     0     0 ...    39 50192  1103]]


In [213]:
EMBEDDING_DIM = 100

In [248]:
combined_index = {**word_index, **word_index_test}

In [255]:
##lets define a new model here that combines the two datasets
def doc2vec_combined_model():
    
    ##We have a 2D representation for the tensor, reduce to 1D which is what we need
    ##first dimension will have variable size and second dimension is 2493

    input_layer = Input(shape=(X_train.shape[1],))


    #Embedding layer
    ##word_index is a dictionary, add +1 for the padding token
    embedding_layer = Embedding(input_dim=len(combined_index) + 1, output_dim=EMBEDDING_DIM)(input_layer)

    #GlobalAveragePooling1D layer
    ##average_pooling = GlobalAveragePooling1D()(embedding_layer)

    #Flattening
    flatten = Flatten()(embedding_layer)

    # Additional Dense layers with dropout for regularization
    dense_1 = Dense(64, activation='relu')(flatten)
    dropout_1 = Dropout(0.2)(dense_1)

    dense_2 = Dense(32, activation='relu')(dropout_1)
    dropout_2 = Dropout(0.1)(dense_2)


    #Dense layers
    #Optimization such as dropout??

    output_layer = Dense(1, activation='sigmoid')(dropout_2)


    model = Model(inputs=input_layer, outputs=output_layer)

    #binary or multi-class compilation
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [256]:
from sklearn.model_selection import train_test_split
print(type(y))
print(type(bin_list))
print(y.shape)
print(bin_list.shape)

combined_label = pd.concat([y, bin_list], ignore_index=True, axis=0)
print(combined_label.shape)

X_train, X_test, y_train, y_test = train_test_split(combined_sequences, combined_label, test_size=0.4, random_state=22)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
(50000,)
(25000,)
(75000,)


In [257]:
##run that model
##Train actual model
model = doc2vec_combined_model()
model.fit(X_train, y_train, batch_size=10, epochs=2, verbose=1)

Epoch 1/2
4500/4500 [==============================] - 544s 121ms/step - loss: 0.4572 - accuracy: 0.7645
Epoch 2/2
4500/4500 [==============================] - 553s 123ms/step - loss: 0.1766 - accuracy: 0.9330


In [258]:
##evaluate new model
# Evaluate the model on the test set
results = model.evaluate(X_test, y_test, batch_size=10)

# Print the evaluation results
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])

3000/3000 [==============================] - 17s 6ms/step - loss: 0.3567 - accuracy: 0.8594
Test Loss: 0.3566715121269226
Test Accuracy: 0.8593999743461609
